In [1]:
"""
    Nome - RGA:
    Fábio Holanda Saraiva Júnior - 2015.1905.006-2
    Felipe Salles Lopez - 2016.1907.032-4
    Lucas Avanzi - 2016.1907.024-3
    Lucas Antonio dos Santos - 2016.1907.013-8
"""

import csv
#import pandas as pd
import numpy as np


from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier


from sklearn.model_selection import KFold


dataset_path = [
                "./datasets/arrhythmia/arrhythmia.data",
                "./datasets/balance-scale/balance-scale.data",
                "./datasets/blood-transfusion/transfusion.data",
                "./datasets/echocardiogram/echocardiogram.data",
                "./datasets/glass/glass.data",
                "./datasets/haberman/haberman.data",
                "./datasets/hepatitis/hepatitis.data",
                "./datasets/iris/iris.data",
                "./datasets/lymphography/lymphography.data",
                "./datasets/tae/tae.data"
                ]
first_class_line = [
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0
]

class_index = [
    279,
    0,
    4,
    12,
    10,
    3,
    0,
    4,
    0,
    5
]

def read_input_file(file_path):
    open_file = open(file_path, "r")
    reader = csv.reader(open_file)
    data = list(reader)

    return data


/home/trevor/Documentos/2020.1/IA/trab2/entrega/py36/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
"""
Base de dados de arritmia cardíaca
"""

#leitura do dataset
dataset_0 = read_input_file(dataset_path[0])

#len(dataset_0[0])


    

In [3]:
"""
Base de dados de peso e distância da balança
"""

#leitura do dataset
dataset_1 = read_input_file(dataset_path[1])

#dataset_1[0]

In [4]:
"""
Dados de centro de serviços de transfusão de sangue
"""

#leitura do dataset
dataset_2 = read_input_file(dataset_path[2])

#dataset_2[0]
#dataset_2[1]

In [5]:
"""
Dados de ecocardiograma
"""

#leitura do dataset
dataset_3 = read_input_file(dataset_path[3])

In [6]:
"""
Banco de dados de identificação de vidro
"""

#leitura do dataset
dataset_4 = read_input_file(dataset_path[4])

In [7]:
"""
Banco de dados de Haberman
"""

#leitura do dataset
dataset_5 = read_input_file(dataset_path[5])

In [8]:
"""
Banco de dados hepatite
"""

#leitura do dataset
dataset_6 = read_input_file(dataset_path[6])

In [29]:
"""
Banco de dados iris
"""

#leitura do dataset
#dataset_7 = read_input_file(dataset_path[7])
pd_dataset_7 = pd.read_csv(dataset_path[7], header = None)
array_7 = np.array(pd_dataset_7)


#Obter apenas os dados e gerar a lista de dados:
#array_7[:, 0:4].tolist()

#Obter classes e gerar listas de classe:
#array_7[:, 4].tolist()


mlp = MLPClassifier(hidden_layer_sizes = (5,), activation="logistic", max_iter = 100, alpha = 0.001, solver = "sgd",
                   tol = 1e-9, learning_rate_init=.01, verbose = True)

data = array_7[:, 0:4]
target = array_7[:, 4]

mlp = mlp.fit (data, target)


Iteration 1, loss = 1.30522282
Iteration 2, loss = 1.29872744
Iteration 3, loss = 1.28975798
Iteration 4, loss = 1.27884640
Iteration 5, loss = 1.26650598
Iteration 6, loss = 1.25321397
Iteration 7, loss = 1.23939974
Iteration 8, loss = 1.22543799
Iteration 9, loss = 1.21164612
Iteration 10, loss = 1.19828489
Iteration 11, loss = 1.18556114
Iteration 12, loss = 1.17363172
Iteration 13, loss = 1.16260763
Iteration 14, loss = 1.15255824
Iteration 15, loss = 1.14351539
Iteration 16, loss = 1.13547762
Iteration 17, loss = 1.12841458
Iteration 18, loss = 1.12227192
Iteration 19, loss = 1.11697632
Iteration 20, loss = 1.11244087
Iteration 21, loss = 1.10857013
Iteration 22, loss = 1.10526498
Iteration 23, loss = 1.10242684
Iteration 24, loss = 1.09996123
Iteration 25, loss = 1.09778047
Iteration 26, loss = 1.09580570
Iteration 27, loss = 1.09396806
Iteration 28, loss = 1.09220931
Iteration 29, loss = 1.09048183
Iteration 30, loss = 1.08874819
Iteration 31, loss = 1.08698046
Iteration 32, los

/home/trevor/Documentos/2020.1/IA/trab2/entrega/py36/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [10]:
"""
Dataset Domínio Linfográfico

Citação: 
This lymphography domain was obtained from the University Medical Centre
Institute of Oncology, Ljubljana, Yugoslavia.  Thanks go to M. Zwitter and 
   M. Soklic for providing the data.
"""

#leitura do dataset
dataset_8 = read_input_file(dataset_path[8])


In [11]:
"""
Dataset Avaliação Assistente de Ensino
"""

#leitura do dataset
dataset_9 = read_input_file(dataset_path[9])